# [Selenium](https://selenium-python.readthedocs.io/)
* [doc offic](https://www.selenium.dev/), [Guide de survie 1](http://akul.me/blog/2016/selenium-cheatsheet/), [Guide de survie 2](http://allselenium.info/python-selenium-commands-cheat-sheet-frequently-used/)
## Installation
* Télécharger [webdriver](https://chromedriver.chromium.org/downloads), Rangez-le dans un dossier et Rajoutez le path de l'executable `chromedriver.exe` dans les variables d'environnement système (puis redémarrer).
### Tuto : 
* Comment identifier les diff élèments [locator in selenium](https://www.lambdatest.com/blog/locators-in-selenium-webdriver-with-examples/), Comment attendre qu'un élément web soit present, visible, cliquable [article](http://allselenium.info/wait-for-elements-python-selenium-webdriver/), [stackoverflow](https://stackoverflow.com/questions/59130200/selenium-wait-until-element-is-present-visible-and-interactable)   
* Scraper des pages dynamiques à l'aide de selenium et BeautifulSoup à la fois : tuto [medium](https://medium.com/ymedialabs-innovation/web-scraping-using-beautiful-soup-and-selenium-for-dynamic-page-2f8ad15efe25) ou [stackoverflow](https://stackoverflow.com/questions/59409418/how-to-rotate-selenium-webrowser-ip-address)
* <https://stackoverflow.com/questions/54862426/python-selenium-get-href-value/54862902>


In [2]:
!pip install selenium

In [1]:
# https://pytest-html.readthedocs.io/en/latest/index.html
!pip install pytest-html

In [2]:
# https://github.com/pytest-dev/pytest-xdist
!pip install pytest-xdist

In [2]:
!pip install pytest-cov

In [2]:
# locust : framework de test de performance facile à utiliser et open source
# https://locust.io
# https://docs.locust.io/en/stable/index.html
# Après l'install du pkg, pensez à rajouter le path aux var d'env (C:\Users\bejao\AppData\Roaming\Python\Python36\Scripts)
!pip install --user locust

  Using cached locust-2.5.0-py3-none-any.whl (795 kB)
  Using cached ConfigArgParse-1.5.3-py3-none-any.whl (20 kB)
  Using cached Flask_BasicAuth-0.2.0-py3-none-any.whl
  Using cached geventhttpclient-1.5.3-cp36-cp36m-win_amd64.whl (36 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


 ## Charger les librairies

In [1]:
# Le module selenium.webdriver fournit toutes les implémentations de WebDriver : Firefox, Chrome, Edge, ... (sont actuellement prises en charge). 
from selenium import webdriver

# La classe Keys fournit des touches dans le clavier telles que RETURN, F1, ALT, etc.
from selenium.webdriver.common.keys import Keys
# La classe ActionsChains permet la combinaison de touches, par ex 4 fois la touche Tab 
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By

import time
import random

# import pytest

# Aujourd'hui, nous allons scrapper et tester plusieurs sites avec Selenium

* Selenium est un outil crée initialement pour automatiser des tests sur des sites web. Il est donc bien utile lorsque des informations sont accesibles en cliquant sur des liens. Un boutton, par exemple, est un élément sur lequel il trés diffcile d'obtenir le lien. BeautifullSoup devient alors limité. Dans ce cas, utilisez Selenium !

# Lancer des tests sur diff browsers : `Chrome`, `FireFox`, `Edge` (peu importe la version) ? 
* [webdriver manager](https://github.com/SergeyPirogov/webdriver_manager) installera dans `C:\Users\bejao\.wdm\drivers` et màj les drivers automq.

In [4]:
!pip install webdriver_manager

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

## Options pour accélerer `webdriver` 

In [7]:
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': { 'images': 2,
                            'plugins': 2, 'popups': 2, 'geolocation': 2, 
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2, 
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2, 
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2, 
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2, 
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2, 
                            'durable_storage': 2}}

options.add_experimental_option("prefs", prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--ignore-certificate-errors')

* Si vs rencontrez ce bug lors du lancement du driver chrome, ajoutez les options suivantes : 
* crbug.com/1216328: Checking Bluetooth availability started. Please report if there is no report that this ends.
[stackoverflow](https://stackoverflow.com/questions/69441767/error-using-selenium-chrome-webdriver-with-python)

In [8]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [9]:
chrome = webdriver.Chrome(ChromeDriverManager().install())

## Quelques options pr contrôller ChromeDriverManager  
# chrome = webdriver.Chrome(ChromeDriverManager(log_level=0, print_first_line=False, cache_valid_range=7).install(), options=options)
# log_level=0 (silent log), print_first_line=False (un saut de ligne est par défaut) => console moins encombrée ... !
# cache_valid_range=7 => augmenter la durée de validité du cache à 7 jours (par défaut 1 jour)
# chrome.implicitly_wait(10)
# chrome.delete_all_cookies()



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\bejao\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache
C:\Users\bejao\miniconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


# Nous allons simuler et tester une recherche sur le site officiel de Python

## Trouver un element par `CSS ID`
* Comment inspecter correctement une page : [tuto](https://www.lambdatest.com/blog/locators-in-selenium-webdriver-with-examples/)
* Inspecter la page et vérifier ceci : 
`<input id="id-search-field" name="q" type="search" role="textbox" class="search-field" placeholder="Search" value="" tabindex="1" xpath="1">`

In [13]:
# C'est la méthode la plus simple ! CSS ID est unique !
# driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

# La méthode driver.get mènera à une page donnée par l'URL. WebDriver attendra que la page soit complètement chargée. Càd que l'événement “onload” s'est déclenché avant de rendre le contrôle à votre script.
driver.get("https://www.python.org")
# Je teste si Python se trouve dans le titre de la page ? 
assert "Python" in driver.title

# Je teste si Java se trouve dans le titre de la page ?
# Une exception est levée : AssertionError (Java n'est pas ds le titre) 
# assert "Java" in driver.title

search_bar = driver.find_element_by_id("id-search-field")
# Pour des raisons de sécurité, nous effacerons tout texte pré-rempli dans le champ de saisie 
# (par exemple, «Rechercher») afin que cela n'affecte pas nos résultats de recherche:
search_bar.clear()
time.sleep(round(random.uniform(3, 5), 1))
search_bar.send_keys("pycon")
time.sleep(round(random.uniform(3, 5), 1))
search_bar.send_keys(Keys.ENTER)

# print(type(driver.find_element_by_id("id-search-field").clear()))
# driver.find_element_by_id("id-search-field").clear().send_keys("pycon" + Keys.ENTER)

# Après soumission de la page, vous devriez obtenir le résultat s'il y en a. Pour vous assurer que certains résultats sont trouvés, faites une assertion:
# assert "No results found." not in driver.page_source

# time.sleep(round(random.uniform(3, 5), 1))

driver.close()
driver.quit()



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\bejao\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache
C:\Users\bejao\miniconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\bejao\miniconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [ ]:
def test_search_bar():
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
    driver.get("https://www.python.org")
    assert "Python" in driver.title


     

In [ ]:
# C'est une 2è méthode qui consiste à chercher par le nom de la classe ! 
# Att : le nom de la classe n'est pas unique parcq par déf la CSS classe est appliquée à un groupe d'élements! 

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get("https://www.python.org")
first_search_bar = driver.find_element_by_class_name("search-field")
first_search_bar.clear()
first_search_bar.send_keys("pycon")
time.sleep(round(random.uniform(3, 5), 1))
first_search_bar.send_keys(Keys.ENTER)
time.sleep(round(random.uniform(3, 5), 1))
driver.close()
driver.quit()

## Trouver un element par `CSS Class Name`

## Trouver un element par `Name`

In [15]:
# Att : le nom de l'élément peut ne pas être unique
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
# driver.get("https://www.python.org")
# assert "Python" in driver.title
# first_search_bar = driver.find_element_by_name("q")
# first_search_bar.clear()
# first_search_bar.send_keys("pycon")
# # time.sleep(round(random.uniform(3, 5), 1)) first_search_bar.send_keys(Keys.ENTER)
# # time.sleep(round(random.uniform(3, 5), 1))
# driver.close()
# driver.quit()

## Trouver un element par `XPath`

In [38]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get("https://www.python.org")
first_search_bar = driver.find_element_by_xpath("//input[@name='q']")
first_search_bar.clear()
first_search_bar.send_keys("pycon")
time.sleep(round(random.uniform(3, 5), 1))
first_search_bar.send_keys(Keys.ENTER)
time.sleep(round(random.uniform(3, 5), 1))
driver.close()
driver.quit()

## Trouver un element par `Tag Name`

In [39]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get("https://www.python.org")
first_search_bar = driver.find_element_by_tag_name("input")
first_search_bar.clear()
first_search_bar.send_keys("pycon")
time.sleep(round(random.uniform(3, 5), 1))
first_search_bar.send_keys(Keys.ENTER)
time.sleep(round(random.uniform(3, 5), 1))
driver.close()
driver.quit()

## Trouver plusieurs élements

In [ ]:
from selenium.webdriver.common.by import By
all_inputs = driver.find_elements(By.XPATH, '//form[@id='loginForm']/input')

# Démo 1

In [14]:
# driver = webdriver.Chrome(executable_path=r'C:/chromedriver.exe')
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())

driver.get('https://www.zalando.fr/')

# Il est à noter que si votre page utilise beaucoup d'AJAX au chargement, WebDriver peut ne pas savoir quand il s'est complètement chargé:
time.sleep(round(random.uniform(3, 5), 1))
# ou encore
# driver.implicitly_wait(10)



# Agrandir, minimiser, full screen la fenêtre
driver.maximize_window()
time.sleep(round(random.uniform(3, 5), 1))
driver.minimize_window()
time.sleep(round(random.uniform(3, 5), 1))
driver.maximize_window()
driver.fullscreen_window()

print(driver.title)
# assert ...
print(driver.current_url)


# WebDriver propose plusieurs méthodes pour rechercher des éléments à l'aide de l'une des méthodes find_element_by_ *  
# Ensuite, nous envoyons des clés. C’est similaire à la saisie de touches à l’aide de votre clavier. 
# Des clés spéciales peuvent être envoyées à l'aide de la classe Keys importée de selenium.webdriver.common.keys 
# search_bar_xpath = "//input[@placeholder='Rechercher']" #  "//input[@name='q']"
# driver.find_element_by_xpath(search_bar_xpath).send_keys("nike")  # 1è façon 
# driver.find_element(By.XPATH, search_bar_xpath).send_keys("nike") # 2è façon (plus élégante)
# driver.find_element(By.XPATH, search_bar_xpath).send_keys("nike" + Keys.ENTER) 

# Q : Cmnt simuler une sasie au clavier (par un humain) ?   
# https://stackoverflow.com/questions/28132165/can-selenium-webdriver-make-a-random-time-beween-send-keys-to-simulate-human-be

# Q : Cmnt saisir plusieurs touches successivement ?  
# https://www.it-swarm-fr.com/fr/java/comment-appuyer-sur-tab-puis-sur-enter-dans-selenium-webdriver-en-utilisant-java/1051914028/

# https://stackoverflow.com/questions/35385221/send-multiple-tab-key-presses-with-selenium


# beaute_xpath = "//span[contains(text(),'Beauté')]"
# rasage_xpath = "//span[normalize-space()='Rasage']"
# # homme_x_path = "(//span[contains(text(),'Homme')])[1]"
# driver.find_element_by_xpath(beaute_xpath).click()
# time.sleep(round(random.uniform(3, 5), 1))
# driver.find_element_by_xpath(rasage_xpath).click()
# time.sleep(round(random.uniform(3, 5), 1))
print(driver.current_url)
# driver.back() 
# driver.find_element_by_xpath(homme_x_path).click()
# time.sleep(round(random.uniform(3, 5), 1))
# print(driver.current_url)
# driver.forward() 
# time.sleep(round(random.uniform(3, 5), 1))
# print(driver.current_url)
# driver.refresh


# time.sleep(round(random.uniform(3, 6), 1))
driver.close() # On ferme le browser
driver.quit()  # on arrete le driver, càd j'allège les ressources   




====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\bejao\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache
C:\Users\bejao\miniconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


Boutique de mode en ligne : Zalando
https://www.zalando.fr/
https://www.zalando.fr/


In [ ]:
address = 'Anthony Street 35'
for letter in address:
    dodo=round(random.uniform(0,1), 2)
    print(dodo)
    time.sleep(dodo)  # sleep between 1 and 3 seconds
    print(letter)

1.31
A
1.64
n
1.77
t
1.46
h
1.8
o
1.93
n
1.64
y
1.89
 
1.15
S
1.7
t
1.48
r
1.68
e
1.44
e
1.36
t
1.36
 
1.49
3
1.35
5


In [ ]:
# Il ne faut pas confondre Selenium et webbrowser !
import webbrowser   #  webbrower Comes with Python and opens a browser to a specific page
webbrowser.open('http://inventwithpython.com/')

# Cl : Selenium : Launches and controls a web browser. Selenium is able to fill in forms and simulate mouse clicks in this browser.

True

# Réccupérer le numéro de téléphone d'une annonce sur [leboncoin](https://www.leboncoin.fr/)

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

url='https://www.leboncoin.fr/sports_hobbies/1536839557.htm/'

driver = webdriver.chrome()
driver.implicitly_wait(10)
driver.get(url)


python_button = driver.find_elements_by_xpath('//div[@data-reactid="269"]')[0]
python_button.click()

# Et après, c'est comme avec Beautiful soup
soup=BeautifulSoup(driver.page_source)

driver.close()

# Et après, c'est comme avec Beautiful soup
for elem in soup.find_all('a',attrs={"data-qa-id" :"adview_number_phone_contact" }):
    print(elem.text)


#### En partant d'une annonce du bon coin, recuperer tous les éléments disponibles permettant de bien définir le produit mis en vente. Utilisez selenium pour le numéro de telephone

# Exercice

 Scrappez la marque, le modèle et le prix d'une série d'articles de votre choix pour en faire un dataframe pandas

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.zalando.fr/')
time.sleep(3)
marques = []
modeles = []
prix = []

articles = browser.find_elements_by_tag_name('article')

for i in range(len(articles)):
    articles = browser.find_elements_by_tag_name('article')
    articles[i].click()
    time.sleep(2)
    print(i)
    
    try:
        marque = browser.find_element_by_xpath('/html/body/div[4]/div/div[1]/div/div[2]/x-wrapper-re-1-3/div[1]/a/span/h3')
        marques.append(marque.text)
    except:
        marques.append('nan')
        pass
    
    try:
        model = browser.find_element_by_xpath('/html/body/div[4]/div/div[1]/div/div[2]/x-wrapper-re-1-3/h1')
        modeles.append(model.text)
    except:
        modeles.append('nan')
        pass
    
    try:
        montant = browser.find_element_by_xpath('/html/body/div[4]/div/div[1]/div/div[2]/x-wrapper-re-1-3/div[3]/div/div/span[1]')
        montant = montant.text.split(' ')
        montant = montant[0]
        prix.append(montant)
    except:
        prix.append('nan')
        pass
    
    browser.back()
    time.sleep(3)

df = pd.DataFrame({'marque': marques, 'modele': modeles, 'prix': prix})

# Exercice

 Scrappez des commentaires sur un site de votre choix (amazon, trip advisor, la fourchette ...) pour en faire un dataset pour faire du NLP (analyse de sentiment, topic modeling, ...)

In [ ]:
chrome.get('https://www.zalando.fr/')
chrome.maximize_window()
print(chrome.title)
time.sleep(3)
chrome.close()
chrome.quit()